In [5]:
!pwd
!pip install --upgrade pip

# Install required libraries
!python3 -m pip -q install redis
!pip install -U langchain gradio
!pip install -U langchain-core
!pip install -U langchain-google-vertexai
!pip install -U langchain-community
%pip install -qU pypdf

/content


In [6]:
## Update the 'host' field with the correct Redis host URL
host = 'redis-16533.c259.us-central1-2.gce.redns.redis-cloud.com'
port = 16533
password = 'admin'
requirePass = True


In [7]:
import redis

if requirePass:
    client = redis.Redis(host = host, port=port, decode_responses=True, password=password)
else:
    client = redis.Redis(host = 'localhost', decode_responses=True)

print(client.ping())
# Clear Redis database (optional)
client.flushdb()

REDIS_URL = f"redis://:{password}@{host}:{port}"
INDEX_NAME = f"idx_qna"

True


In [3]:
!wget https://storage.googleapis.com/abhi-data-2024/how_india_shops_online.pdf -O report.pdf


--2025-01-14 14:52:18--  https://storage.googleapis.com/abhi-data-2024/how_india_shops_online.pdf
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13218601 (13M) [application/pdf]
Saving to: ‘report.pdf’

report.pdf          100%[===================>]  12.61M  5.23MB/s    in 2.4s    

2025-01-14 14:52:22 (5.23 MB/s) - ‘report.pdf’ saved [13218601/13218601]



In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA

#from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader


file = "report.pdf"

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=50, add_start_index=True
)

loader = PyPDFLoader(file)
documents = loader.load()

chunks = text_splitter.split_documents(documents)
#chunked_docs = [doc.page_content for doc in chunks]

In [9]:
chunked_docs = [doc.page_content for doc in chunks]
print(len(chunks))
print(len(chunked_docs))

62
62


In [13]:
from langchain_google_vertexai import VertexAIEmbeddings
from google.colab import auth
from getpass import getpass
import vertexai
import google.auth
import vertexai

# input your GCP project ID and region for Vertex AI
PROJECT_ID = 'central-beach-194106'#getpass("PROJECT_ID:")
REGION = 'us-central1' #input("REGION:")

creds, _ = google.auth.default(quota_project_id=PROJECT_ID)
print('Authenticated')

vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')
print(creds)


PROJECT_ID:··········
Authenticated
PROJECT_ID: central-beach-194106 & REGION: us-central1


# Create text embeddings with Vertex AI embedding model

Use the Vertex AI API for text embeddings, developed by Google.

Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:


*   Semantic search: Search text ranked by semantic similarity.
*   Recommendation: Return items with text attributes similar to the given text.
*   Classification: Return the class of items whose text attributes are similar to the given text.
*   Clustering: Cluster items whose text attributes are similar to the given text.
*   Outlier Detection: Return items where text attributes are least related to the given text.

The Vertex AI text-embeddings API lets you create a text embedding using Generative AI on Vertex AI. The textembedding-gecko model accepts a maximum of 3,072 input tokens (i.e. words) and outputs 768-dimensional vector embeddings.

In [16]:
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA
from langchain_google_vertexai import VertexAIEmbeddings

from langchain.document_loaders import UnstructuredFileLoader

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

def get_vectordb() -> Redis:
    """Create the Redis vectordb."""
    # Load Redis with documents
    vectordb = Redis.from_documents(
        documents=chunks,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectordb


redis = get_vectordb()

#embedder = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x79aa4d604880>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

KeyboardInterrupt: 

# Include RAG

We're going to build a complete RAG pipeline from scratch incorporating the following components:

Standard retrieval and chat completion
Dense content representation to improve accuracy
Query re-writing to improve accuracy
Semantic caching to improve performance
Conversational session history to improve personalization

### Define Prompt template
PromptTemplate defines the exect text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [ ]:
#@title Function to define prompt template

def create_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use only the following pieces of context to answer the question. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt


In [ ]:
#@title Invoke Google Vertex LLM using Langchain
# This is where the Langchain brings all the components together in a form of a simple QnA chain
from langchain_google_vertexai import VertexAI

llm = VertexAI(
    model_name="gemini-1.5-pro-002",
    max_output_tokens=2048,
    temperature=0.5,
    verbose=False,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=redis.as_retriever(search_type="similarity_distance_threshold",search_kwargs={"distance_threshold":0.5}),
    #return_source_documents=True,
    chain_type_kwargs={"prompt": create_prompt()},
    #verbose=True
    )

In [ ]:
qa.invoke('What are some motivations for shopping online?')['result']

'Question: What are some motivations for shopping online?\nAnswer: Some motivations for online shopping include the absence of physical stores for premium brands, stockouts of certain products, lack of knowledgeable staff in offline stores, lack of discounts and special offers in physical stores, large crowds in malls during weekends, and browsing through online shopping platforms during free time.\n'

In [ ]:
qa.invoke('How do Indians like to pay for shopping online?')['result']

'Question: How do Indians like to pay for shopping online?\nAnswer: Urban dwellers and rest of India consumers show similar acceptance levels for UPI payments. However, cash on delivery (CoD) is still preferred among rest of India consumers to minimize fraud risk.  Generation X in rest of India prefers card transactions for mid-high value purchases on well-known platforms. Paytm is popular among urban dwellers, while PhonePe is preferred by rest of India. Google Pay ranks second nationwide.\n'

In [ ]:
qa.invoke('What are some known challenges in shopping online?')['result']

'Question: What are some known challenges in shopping online?\nAnswer: Some known challenges of shopping online are payment fraud, credibility of unfamiliar websites, and doubts regarding product quality matching the images shown.  Also, urban delivery faces logistical overload, regulations, and access restrictions, causing inconsistent experiences to the consumers. \n'

In [ ]:
qa.invoke('How home and kitchen segment is growing?')['result']

'Question: How home and kitchen segment is growing?\nAnswer: In the Indian market, the home furnishing and kitchen sector is growing at a CAGR of 10%. \n'

In [ ]:
qa.invoke('What are the effects of social media on online shopping?')['result']

'Answer: Social media has amplified awareness and aspirations of products. 62% of respondents tried products after seeing them on Facebook and Instagram. Social media is the most preferred channel for encouraging trials of new products. \n'

In [ ]:
qa.invoke('What are some relevant items that are shopped online?')['result']

'    Answer: Grocery, Fashion and Accessories, Electronics and Consumer Durables, Beauty and Personal Care, Health and Wellness, Home and Kitchen, Sports and Fitness \n'

In [ ]:
import gradio as gr

def handle(query):
    response = qa.run(query)
    return response

iface = gr.Interface(fn=handle, inputs="text", outputs="text")
iface.launch(share=True)

In [ ]:
iface.close()

Closing server running on port: 7860
